# Neural Network Model

## Table of Contents

- [Data Takeover](#Data-Takeover)
    - [Train/Test Split](#Train/Test-Split)
- [Neural Network Implementation](#Neural-Network-Implementation)
    - [Performance Measurement](#Performance-Measurement)

## Data Takeover

Read in DataFrame from chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb) as input for processing in this chapter.

In [1]:
import os
import pandas as pd

path_goldstandard = './daten_goldstandard'

# Restore results so far
df_labelled_feature_matrix = pd.read_pickle(os.path.join(path_goldstandard,
                                                         'labelled_feature_matrix.pkl'),
                                 compression=None)

df_labelled_feature_matrix.head()

,duplicates,century_delta,edition_delta,format_prefix_delta,format_postfix_delta,person_245c_delta,ttlfull_245_delta,ttlfull_246_delta,volumes_delta
0,1,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0
1,1,1.0,1.0,1.0,1.0,0.818905,0.363636,1.0,1.0
2,1,1.0,1.0,1.0,1.0,0.697740,1.000000,1.0,1.0
3,1,1.0,1.0,1.0,1.0,0.818905,0.363636,1.0,1.0
4,1,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0


In [2]:
df_labelled_feature_matrix = pd.concat(
    [df_labelled_feature_matrix[df_labelled_feature_matrix.duplicates==0].sample(n=1473),
     df_labelled_feature_matrix[df_labelled_feature_matrix.duplicates==1]])

In [3]:
print('Part of duplicates (1) on uniques (2) in units of [%]')
print(df_labelled_feature_matrix.duplicates.value_counts(normalize=True)*100)

Part of duplicates (1) on uniques (2) in units of [%]
1    50.0
0    50.0
Name: duplicates, dtype: float64


### Train/Test Split

The train/test split will be implemented here as a general function to be called in the models chapters.

In [4]:
X = df_labelled_feature_matrix.drop(columns=['duplicates']).values
y = df_labelled_feature_matrix.duplicates.values

In [5]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)

In [6]:
X_tr.shape

(2356, 8)

## Neural Network Implementation

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.initializers import VarianceScaling
from keras.regularizers import l2
from keras.optimizers import SGD

l2_alpha = 0.1
sgd_learnrate = 0.005

# Input layer
hidden = Dense(
    12, # Number of units for hidden layer
    input_shape=(X_tr.shape[1],),
    activation='relu',
    kernel_initializer=VarianceScaling(scale=2.0, seed=0),
    kernel_regularizer=l2(l2_alpha),
    bias_initializer='zeros'
)

# With dropout layer
dropout = Dropout(
    rate=0.25,
    seed=0
)

# Output layer
output = Dense(
    2,
    activation='sigmoid', # Bring out 0 or 1 values
    kernel_initializer=VarianceScaling(scale=1, seed=0),
    kernel_regularizer=l2(l2_alpha),
    bias_initializer='zeros'
)

# Create model with sequential API
model = Sequential()
model.add( hidden ) # Hidden layer
model.add( dropout ) # Dropout
model.add( output ) # Output layer

# The optimizer is to be SGD
sgd = SGD(lr=sgd_learnrate)

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy']
#    metrics=['categorical_accuracy']
)

# Check Model configuration
model.get_config()

{'name': 'sequential_5',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_9',
    'trainable': True,
    'batch_input_shape': (None, 8),
    'dtype': 'float32',
    'units': 12,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 2.0,
      'mode': 'fan_in',
      'distribution': 'normal',
      'seed': 1}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': {'class_name': 'L1L2',
     'config': {'l1': 0.0, 'l2': 0.10000000149011612}},
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dropout',
   'config': {'name': 'dropout_5',
    'trainable': True,
    'dtype': 'float32',
    'rate': 0.25,
    'noise_shape': None,
    'seed': 0}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_10',
    'trainable': True,
    'dtype': 'float32',
    'units': 2,
    'a

In [35]:
from keras.utils import to_categorical

# Parameters for fitting with batches and epochs
epochs = 100
batch_size = 50
# Stay with the four logits and do not reduce to one categorical integer
y_tr_2 = to_categorical(y_tr)
y_val_2 = to_categorical(y_te)

# Fit the model
history = model.fit(
    x=X_tr, y=y_tr_2,
    batch_size=batch_size, epochs=epochs,
    validation_data=(X_te, y_val_2)
)

Train on 2356 samples, validate on 590 samples
Epoch 1/100
2356/2356 [==============================] - 1s 359us/step - loss: 3.9343 - accuracy: 0.5314 - val_loss: 3.7447 - val_accuracy: 0.5288
Epoch 2/100
2356/2356 [==============================] - 0s 58us/step - loss: 3.6181 - accuracy: 0.5480 - val_loss: 3.4554 - val_accuracy: 0.5475
Epoch 3/100
2356/2356 [==============================] - 0s 59us/step - loss: 3.3435 - accuracy: 0.5581 - val_loss: 3.1960 - val_accuracy: 0.5593
Epoch 4/100
2356/2356 [==============================] - 0s 52us/step - loss: 3.0935 - accuracy: 0.5700 - val_loss: 2.9615 - val_accuracy: 0.5763
Epoch 5/100
2356/2356 [==============================] - 0s 60us/step - loss: 2.8712 - accuracy: 0.5908 - val_loss: 2.7483 - val_accuracy: 0.5847
Epoch 6/100
2356/2356 [==============================] - 0s 89us/step - loss: 2.6633 - accuracy: 0.5938 - val_loss: 2.5550 - val_accuracy: 0.5949
Epoch 7/100
2356/2356 [==============================] - 0s 48us/step - loss

2356/2356 [==============================] - 0s 49us/step - loss: 0.6832 - accuracy: 0.8531 - val_loss: 0.6738 - val_accuracy: 0.9356
Epoch 57/100
2356/2356 [==============================] - 0s 45us/step - loss: 0.6817 - accuracy: 0.8455 - val_loss: 0.6721 - val_accuracy: 0.9390
Epoch 58/100
2356/2356 [==============================] - 0s 48us/step - loss: 0.6782 - accuracy: 0.8485 - val_loss: 0.6704 - val_accuracy: 0.9373
Epoch 59/100
2356/2356 [==============================] - 0s 46us/step - loss: 0.6782 - accuracy: 0.8561 - val_loss: 0.6689 - val_accuracy: 0.9390
Epoch 60/100
2356/2356 [==============================] - 0s 48us/step - loss: 0.6752 - accuracy: 0.8561 - val_loss: 0.6674 - val_accuracy: 0.9390
Epoch 61/100
2356/2356 [==============================] - 0s 47us/step - loss: 0.6763 - accuracy: 0.8455 - val_loss: 0.6662 - val_accuracy: 0.9424
Epoch 62/100
2356/2356 [==============================] - 0s 48us/step - loss: 0.6757 - accuracy: 0.8497 - val_loss: 0.6649 - val_a

In [36]:
y_tr_2, y_val_2

(array([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]], dtype=float32), array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=float32))

In [37]:
#from keras.utils import plot_model

#plot_model(model, show_shapes=True, dpi=72)

In [38]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.title(r'Bilayer neural network with lr = {} and $\alpha=${}'.format(
    sgd_learnrate, l2_alpha)
)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [39]:
import numpy as np

no_last = 5 # I take the last 5 due to the upwards shape of the curve at its end
print('Mean last {:d} validation accuracy : {:.3f}'.format(
    no_last, np.mean(history.history['val_accuracy'][-no_last:])
))

print('Neural network accuracy (test set): {:.3f}'.format(
    model.evaluate(X_te, to_categorical(y_te),
                   verbose=0)[1] # Loss is at index=0, accuracy at index=1
))

Mean last 5 validation accuracy : 0.955
Neural network accuracy (test set): 0.954


In [40]:
y_pred = model.predict_classes(X_te)

min(y_pred)

0

### Performance Measurement

In [41]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_te, y_pred)

array([[276,  19],
       [  8, 287]])

In [42]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from keras.metrics import accuracy

#print('Score {:.1f}%'.format(100*accuracy_score(X_te, y_te)))
print('Area under the curve {:.1f}% - accuracy {:.1f}% - precision {:.1f}% - recall {:.1f}%'.format(100*roc_auc_score(y_te, y_pred),
                100*accuracy_score(y_te, y_pred),
                100*precision_score(y_te, y_pred),
                100*recall_score(y_te, y_pred)
               ))

Area under the curve 95.4% - accuracy 95.4% - precision 93.8% - recall 97.3%


## Results Handover

In [43]:
# Restore results so far
df_result = pd.read_pickle(os.path.join(path_goldstandard,
                                        'results.pkl'), compression=None)


In [44]:
# Add result of this section
df_result = df_result.append(pd.DataFrame.from_dict({
    'model': ['Neural Network'],
    'test_score' : [0.0],
    'auc' : [100*roc_auc_score(y_te, y_pred)],
    'accuracy' : [100*accuracy_score(y_te, y_pred)],
    'precision' : [100*precision_score(y_te, y_pred)],
    'recall' : [100*recall_score(y_te, y_pred)]
}))

# Save full DataFrame into pickle file
df_result.to_pickle(os.path.join(path_goldstandard,
                                 'results.pkl'), compression=None)